In [1]:
# import tweepy
import pandas as pd
import os
import sys
import requests
import pymongo
from pymongo import MongoClient

# Set Variables

In [2]:
consumer_key = os.getenv("CONSUMER_KEY")
consumer_secret = os.getenv("CONSUMER_SECRET")
access_token = os.getenv("ACCESS_TOKEN")
# access_token_secret = 
os.getenv("ACCESS_TOKEN_SECRET")
bearer_token = os.getenv("BEARER_TOKEN")

In [3]:
client_db =  'stream'
client_col = 'frenchgp' #args.collection

# source and target connections
client = MongoClient('mongodb://localhost:27017')

# database and collection names
db = client[client_db]
col = db[client_col]

# Build Stream

In [6]:
rules = [{"value": '((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc \
OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR \
stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell \
OR mazepin OR latifi) (f1 OR formula1 OR "french grand prix" OR ricard OR \
"french GP" OR "grand prix" OR "formula 1" OR frenchgp)) -indycar \
-"indy car" -is:retweet -is:reply -is:quote'}]
rules2 = [{"value": '(formula1 OR #f1 OR "formula one" OR "formula 1") -indycar \
-"indy car" -is:retweet -is:reply -is:quote'}]

params = {
    'tweet.fields': 'created_at,public_metrics',
    'expansions': 'author_id,referenced_tweets.id,geo.place_id',
    'user.fields': 'created_at,location'
}

In [13]:


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

headers = create_headers(bearer_token)
def getRules():
    response2 = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=create_headers(bearer_token))
    print("\nGETTING RULES: ", response2.text)
    return response2.json()


def deleteRules(response2):

    response2 = getRules()
    ids = [i['id'] for i in response2['data']]
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload)
    print("\nDELETING RULES\n")



def setRules(rules):
    payload = {"add": rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=create_headers(bearer_token),
        json=payload,
    )
    print("\nSET RULES: ", response.text)


def getStream(headers):
    with requests.get("https://api.twitter.com/2/tweets/search/stream",
                      headers=headers,
                      params=params,
                      stream=True) as response:
        print(response)
        for response_line in response.iter_lines():
            if response_line:
                json_response = json.loads(response_line)
                print(json.dumps(json_response, indent=4, sort_keys=True))
                col.insert_one(json.loads(response_line))
                
            

def main():
    headers = create_headers(bearer_token)
    response2 = getRules()
    deleteRules(response2)
    setRules(rules)
    setRules(rules2)
    getRules()
    while True:
        getStream(headers)
        timeout += 1


if __name__ == "__main__":
    main()


GETTING RULES:  {"data":[{"id":"1406261525216731136","value":"((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell OR mazepin OR latifi) (f1 OR formula1 OR \"french grand prix\" OR ricard OR \"french GP\" OR \"grand prix\" OR \"formula 1\" OR frenchgp)) -indycar -\"indy car\" -is:retweet -is:reply -is:quote"},{"id":"1406261530010730502","value":"(formula1 OR #f1 OR \"formula one\" OR \"formula 1\") -indycar -\"indy car\" -is:retweet -is:reply -is:quote"}],"meta":{"sent":"2021-06-19T14:44:45.127Z"}}

GETTING RULES:  {"data":[{"id":"1406261525216731136","value":"((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell OR mazepin OR latifi) (f1 OR formula1 OR \"french grand prix\" OR ricard 

{
    "data": {
        "author_id": "321277002",
        "created_at": "2021-06-19T14:44:54.000Z",
        "geo": {},
        "id": "1406261714878881795",
        "public_metrics": {
            "like_count": 0,
            "quote_count": 0,
            "reply_count": 1,
            "retweet_count": 0
        },
        "text": "#franceGP \ud83c\uddeb\ud83c\uddf7 #F1 Kvalifikace 2021 vs 2019: Ver \ud83c\uddf3\ud83c\uddf11:29.990 vs 1:29.409 (+ 0.581s), Ham \ud83c\uddec\ud83c\udde71:30.248 vs 1:28.319 (+ 1.929s), Bot \ud83c\uddeb\ud83c\uddee1:30.376 vs 1:28.605 (+ 1.771s), Per \ud83c\uddf2\ud83c\uddfd1:30.445 vs 1:30.738 (RP) (- 0.293s), Sai \ud83c\uddea\ud83c\uddf81:30.840 vs 1:29.522 (McLaren) (+ 1.318s)"
    },
    "includes": {
        "users": [
            {
                "created_at": "2011-06-21T09:15:28.000Z",
                "id": "321277002",
                "location": "Petrovice u Karvin\u00e9 Czech Rep.",
                "name": "Jan Schneeberger",
                "user

KeyboardInterrupt: 

In [10]:
headers = create_headers(bearer_token)
response2 = getRules()
deleteRules(response2)
setRules(rules)
setRules(rules2)
getRules()

GETTING RULES:  {"data":[{"id":"1406260610464104450","value":"((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell OR mazepin OR latifi) (f1 OR formula1 OR \"french grand prix\" OR ricard OR \"french GP\" OR \"grand prix\" OR \"formula 1\" OR frenchgp)) -indycar -\"indy car\" -is:retweet -is:reply -is:quote"},{"id":"1406260615392419841","value":"(formula1 OR #f1 OR \"formula one\" OR \"formula 1\") -indycar -\"indy car\" -is:retweet -is:reply -is:quote"}],"meta":{"sent":"2021-06-19T14:41:23.467Z"}}
GETTING RULES:  {"data":[{"id":"1406260610464104450","value":"((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell OR mazepin OR latifi) (f1 OR formula1 OR \"french grand prix\" OR ricard OR

{'data': [{'id': '1406260839292809218',
   'value': '((gasly OR hamilton OR verstappen OR perez OR norris OR leclerc OR bottas OR sainz OR gasly OR vettel OR ricciardo OR alonso OR ocon OR stroll OR tsunoda OR raikkonen OR giovinazzi OR schumacher OR russell OR mazepin OR latifi) (f1 OR formula1 OR "french grand prix" OR ricard OR "french GP" OR "grand prix" OR "formula 1" OR frenchgp)) -indycar -"indy car" -is:retweet -is:reply -is:quote'},
  {'id': '1406260860272656391',
   'value': '(formula1 OR #f1 OR "formula one" OR "formula 1") -indycar -"indy car" -is:retweet -is:reply -is:quote'}],
 'meta': {'sent': '2021-06-19T14:41:32.684Z'}}

In [ ]:
response2 = getRules()
deleteRules(response2)

In [ ]:
r = requests.get(
"https://api.twitter.com/2/tweets/search/stream/rules",
headers=create_headers(bearer_token))
print("GETTING RULES: ", r.text)


# Get Historic

In [ ]:
  
import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def auth():
    return os.environ.get("BEARER_TOKEN")


def create_url():
    tweet_fields = "tweet.fields=lang,author_id"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    ids = "ids=1278747501642657792,1255542774432063488"
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets?{}&{}".format(ids, tweet_fields)
    return url


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    bearer_token = auth()
    url = create_url()
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    print(json.dumps(json_response, indent=4, sort_keys=True))


if __name__ == "__main__":
    main()

## Get User

In [ ]:
import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def auth():
    return bearer_token


def create_url():
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values.
    usernames = "usernames=ik6292" #TwitterDev,TwitterAPI"
    user_fields = "user.fields=description,created_at"
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url = "https://api.twitter.com/2/users/by"
    return url

def create_params():
    params = {"usernames": "ik6292",
              "user.fields": "description,created_at,public_metrics"}
    return params

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, params=create_params(), headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    bearer_token = auth()
    url = create_url()
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    print(json.dumps(json_response, indent=4, sort_keys=True))


if __name__ == "__main__":
    main()